In [1]:
import os
import numpy as np
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Function to get user preferences
def get_user_preferences():
    print("Welcome to the Outfit Recommendation System!")
    print("Please provide your preferences:")
    
    weather = input("Weather (e.g., sunny, rainy, cold): ")
    occasion = input("Occasion (e.g., casual, formal, party): ")
    gender = input("Gender (male/female): ")
    
    return weather, occasion, gender

# Function to load image dataset
def load_image_dataset(dataset_path):
    dataset = {}
    for category in os.listdir(dataset_path):
        category_path = os.path.join(dataset_path, category)
        if os.path.isdir(category_path):
            dataset[category] = [os.path.join(category_path, f) for f in os.listdir(category_path)]
    return dataset

# Function to extract features from images using a trained CNN model
def extract_features_cnn(model, dataset):
    features = []
    for category, image_paths in dataset.items():
        for img_path in image_paths:
            img = image.load_img(img_path, target_size=(img_height, img_width))
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            feature = model.predict(img_array)
            features.append(feature.flatten())
    return np.array(features)

# Function to recommend outfit based on user preferences
def recommend_outfit(weather, occasion, gender):
    # Filter dataset based on user preferences
    filtered_images = []
    for category, image_paths in image_dataset.items():
        if weather in category.lower() and occasion in category.lower() and gender in category.lower():
            filtered_images.extend(image_paths)
    
    # Extract features for filtered images
    filtered_features = extract_features_cnn(cnn_model, {'filtered_images': filtered_images})
    
    # Find nearest neighbors
    distances, indices = knn_model.kneighbors(filtered_features)
    
    # Get recommended outfits
    recommended_outfits = []
    for neighbor_indices in indices:
        for idx in neighbor_indices:
            recommended_outfits.append(filtered_images[idx])
    
    # Count frequency of each recommended outfit
    outfit_counter = Counter(recommended_outfits)
    
    # Return top recommended outfits
    top_outfits = outfit_counter.most_common(5)  # Get top 5 recommended outfits
    return top_outfits

# Define model architecture
def create_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Load and preprocess images using ImageDataGenerator
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

# Get user preferences
weather, occasion, gender = get_user_preferences()

# Load image dataset
dataset_path = 'path/to/your/image/dataset'
image_dataset = load_image_dataset(dataset_path)

# Load pre-trained ResNet50 model for feature extraction
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Extract features from image dataset using the pre-trained ResNet50 model
cnn_features = extract_features_cnn(resnet_model, image_dataset)

# Build KNN model
knn_model = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn_model.fit(cnn_features)

# Define image dimensions and number of classes
img_height, img_width = 150, 150  # Adjust based on your image dataset dimensions
num_classes = len(image_dataset)  # Number of categories/classes

# Create CNN model
cnn_model = create_cnn_model((img_height, img_width, 3))

# Compile CNN model
cnn_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Data preprocessing and augmentation
datagen = ImageDataGenerator(rescale=1./255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             validation_split=0.2)  # Split data into training and validation sets

# Load and preprocess images using ImageDataGenerator
train_generator = datagen.flow_from_directory(dataset_path,
                                              target_size=(img_height, img_width),
                                              batch_size=32,
                                              class_mode='categorical',
                                              subset='training')  # Use training subset
validation_generator = datagen.flow_from_directory(dataset_path,
                                                   target_size=(img_height, img_width),
                                                   batch_size=32,
                                                   class_mode='categorical',
                                                   subset='validation')  # Use validation subset

# Train the CNN model
history = cnn_model.fit(train_generator,
                        steps_per_epoch=train_generator.samples // train_generator.batch_size,
                        validation_data=validation_generator,
                        validation_steps=validation_generator.samples // validation_generator.batch_size,
                        epochs=10)  # Adjust number of epochs as needed

# Evaluate CNN model
loss, accuracy = cnn_model.evaluate(validation_generator)
print("Validation Accuracy:", accuracy)

# Recommend outfit based on user preferences
recommended_outfits = recommend_outfit(weather, occasion, gender)
print("Top 5 recommended outfits:")
for outfit, count in recommended_outfits:
    print(outfit, "-", count, "occurrences")


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
!pip install tensorflow